In [1]:
import tensorflow as tf
import numpy as np

In [2]:
local_opti_epochs = 5
global_opti_epochs = 2
num_tasks = 5

In [3]:
class Model(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.hidden1 = tf.keras.layers.Dense(40, input_shape=(1,))
        self.hidden2 = tf.keras.layers.Dense(40)
        self.out = tf.keras.layers.Dense(1)
        
    def call(self, x):
        x = tf.keras.activations.relu(self.hidden1(x))
        x = tf.keras.activations.relu(self.hidden2(x))
        x = self.out(x)
        return x

In [4]:
def np_to_tensor(list_of_numpy_objs):
    return (tf.convert_to_tensor(obj) for obj in list_of_numpy_objs)

In [5]:
def loss_func(pred_y, y):
    return tf.keras.losses.MSE(y_true, y_pred)

In [6]:
xs = []
for i in range(num_tasks):
    x = np.random.random((1000, 32))
    xs.append(x)
y = np.random.random((1000, 1)) * 3

In [7]:
model = Model()
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

In [8]:
class Global_Model:
    def __init__(self, num_augs):
        self.apply_dist = tf.Variable(initial_value=[1/num_augs] * num_augs, trainable=True)
                    
    def call(self, x, training=True):
        result = self.apply_dist
        if training:
            std_val = tf.math.reduce_std(self.apply_dist)
            std_val += 0.01
            result = result + tf.random.normal(result.shape, mean=0.0, stddev=std_val)
            result = tf.expand_dims(result, axis=0)
            result = tf.random.categorical(tf.math.log(result), 1)[0]
        else:
            result = tf.argmax(result)
        
        return result

        
        


In [9]:
gm = Global_Model(num_tasks)
for ge in range(global_opti_epochs):
    print(gm.call(None, True))
    print(gm.call(None, False))
        

tf.Tensor([0], shape=(1,), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor([2], shape=(1,), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
